# Data Extraction Notebook

This notebook explores function that can scrape and extract separate sections of text from websites relating to 'Privacy Policy' and or 'Terms and Conditions' (this includes 'Terms of Service', 'Terms of Use', etc.)

A key challenge with this work is that different websites have different formats:
- sometimes these sites have numbered section headings (either X. or X.Y)
- sometimes they do not have any numbered section headings and are separated by a html header tag (i.e., h2, h3)
- sometimes the website format is tricky -> div panels, javascript


In [ ]:
# Read in required packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

**For websites that DO NOT have numbered sections and are identified by h3 or h2 tag**
* Does not work on everything

In [120]:
# Given URL return pandas df with a row per section
# Only works on websites where a h3 or h2 tag separates each section

def scrape_and_store_h3_h2_text(url):
    data = []  # List to store extracted data

    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:  # If the request is successful
        soup = BeautifulSoup(response.text, "html.parser")  # Create a BeautifulSoup object 'soup'
        h3_tags = soup.find_all(["h3", "h2"])  # Find all <h3> and <h2> tags
        for h_tag in h3_tags:
            section_text = ""
            next_sibling = h_tag.find_next_sibling()  # Get the next sibling of the <h3> or <h2> tag
            while next_sibling and (next_sibling.name not in ["h3", "h2"]):
                section_text += str(next_sibling)  # Include the entire tag content
                next_sibling = next_sibling.find_next_sibling()  # Move to the next sibling
            if section_text:
                section_text_cleaned = BeautifulSoup(section_text, "html.parser").get_text()  # Clean HTML content
                data.append({"URL": url, "Section": h_tag.get_text(), "Text": section_text_cleaned})
    else:
        print("Failed to retrieve the page:", url)

    df = pd.DataFrame(data)  # Create a DataFrame from the collected data
    return df




In [129]:
# https://help.netflix.com/en/legal/privacy
# https://www.whatsapp.com/legal/privacy-policy
# https://www.airbnb.com.au/help/article/3175 (numbered)

URL = "https://help.instagram.com/155833707900388"

# Example
if __name__ == "__main__":
    url = URL  # Replace with the actual URL
    df = scrape_and_store_h3_h2_text(url)
    if not df.empty:
        pd.set_option("display.max_colwidth", None)  # Display full content of DataFrame columns

In [130]:
# Show result
df

URL  \
0  https://help.instagram.com/155833707900388   
1  https://help.instagram.com/155833707900388   
2  https://help.instagram.com/155833707900388   
3  https://help.instagram.com/155833707900388   
4  https://help.instagram.com/155833707900388   
5  https://help.instagram.com/155833707900388   
6  https://help.instagram.com/155833707900388   
7  https://help.instagram.com/155833707900388   
8  https://help.instagram.com/155833707900388   
9  https://help.instagram.com/155833707900388   

                                                                    Section  \
0                               I. What kinds of information do we collect?   
1                                       II. How do we use this information?   
2                                      III. How is this information shared?   
3                              IV. How do the Meta Companies work together?   
4                       V. How can I manage or delete information about me?   
5                  VI. How do we respond to legal requests or prevent harm?   
6  VII. How do we operate and transfer data as part of our global services?   
7                   VIII. How will we notify you of changes to this policy?   
8                                      IX. How to contact us with questions   
9                                   Privacy Notice for California Residents   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

**For numbered sections (X.)**